In [87]:
from math import log, sqrt, exp
import numpy as np
from scipy.stats import norm
import pandas as pd

interest_rate = 0.03
historical_iv  = 0.262

def greeks(s, k, sigma, r, t, option_type):
    sqrt_t = sqrt(t)
    d1 = (log(s / k) + (r + pow(sigma, 2) / 2.0) * t) / (sigma * sqrt_t)
    d2 = d1 - sigma * sqrt_t
    tmp = exp(-pow(d1, 2) / 2.0)
    tmp2 = sqrt(2.0 * np.pi * t)
    tmp3 = r * k * exp(-r * t)
    gamma = tmp / (s * sigma * tmp2)
    theta_call = -(s * sigma * tmp) / (2.0 * tmp2) - tmp3 * norm.cdf(d2)
    vega = s * sqrt_t * tmp / sqrt(2.0 * np.pi)
    if option_type == 'C':
        delta = norm.cdf(d1)
        theta = theta_call
    else:
        delta = norm.cdf(d1) - 1.0
        theta = theta_call + tmp3
    return delta, gamma, theta, vega


In [88]:
def bs_call(s, k, sigma, r, t):
    tmp = sqrt(t)
    d1 = (log(s / k) + (r + pow(sigma, 2) / 2.0) * t) / (sigma * tmp)
    d2 = d1 - sigma * tmp
    return s * norm.cdf(d1) - k * exp(-r * t) * norm.cdf(d2)


def bs_put(s, k, sigma, r, t):
    tmp = sqrt(t)
    d1 = (log(s / k) + (r + pow(sigma, 2) / 2.0) * t) / (sigma * tmp)
    d2 = d1 - sigma * tmp
    return k * exp(-r * t) * norm.cdf(-d2) - s * norm.cdf(-d1)


def call_iv(c, s, k, t, r=0.03, sigma_min=0.01, sigma_max=1.0, e=0.00001):
    sigma_mid = (sigma_min + sigma_max) / 2.0
    call_min = bs_call(s, k, sigma_min, r, t)
    call_max = bs_call(s, k, sigma_max, r, t)
    call_mid = bs_call(s, k, sigma_mid, r, t)
    diff = c - call_mid
    if c <= call_min:
        return sigma_min
    elif c >= call_max:
        return sigma_max
    while abs(diff) > e:
        if c > call_mid:
            sigma_min = sigma_mid
        else:
            sigma_max = sigma_mid
        sigma_mid = (sigma_min + sigma_max) / 2.0
        call_mid = bs_call(s, k, sigma_mid, r, t)
        diff = c - call_mid
    # print(sigma_mid)
    return sigma_mid

def put_iv(c, s, k, t, r=0.03, sigma_min=0.01, sigma_max=1.0, e=0.00001):
    sigma_mid = (sigma_min + sigma_max) / 2.0
    put_min = bs_put(s, k, sigma_min, r, t)
    put_max = bs_put(s, k, sigma_max, r, t)
    put_mid = bs_put(s, k, sigma_mid, r, t)
    diff = c - put_mid
    if c <= put_min:
        return sigma_min
    elif c >= put_max:
        return sigma_max
    while abs(diff) > e:
        if c > put_mid:
            sigma_min = sigma_mid
        else:
            sigma_max = sigma_mid
        sigma_mid = (sigma_min + sigma_max) / 2.0
        put_mid = bs_put(s, k, sigma_mid, r, t)
        diff = c - put_mid
    return sigma_mid

In [89]:
active_df = ak.option_value_analysis_em()
active_df['expire'] = pd.to_datetime(active_df['到期日'])
active_df

,期权代码,期权名称,最新价,时间价值,内在价值,隐含波动率,理论价格,标的名称,标的最新价,标的近一年波动率,到期日,expire
0,10004274,50ETF沽12月2950,0.2700,0.1130,0.157,22.27,0.2490,上证50ETF,2.793,19.97,2022-12-28,2022-12-28
1,10004273,50ETF购12月2950,0.1495,0.1495,0.000,21.78,0.1330,上证50ETF,2.793,19.97,2022-12-28,2022-12-28
2,10004272,300ETF沽12月4300,0.4630,0.1780,0.285,25.19,0.3836,沪深300ETF,4.015,18.96,2022-12-28,2022-12-28
3,10004271,300ETF沽12月4200,0.3985,0.2135,0.185,24.95,0.3207,沪深300ETF,4.015,18.96,2022-12-28,2022-12-28
4,10004270,300ETF沽12月4100,0.3350,0.2500,0.085,24.45,0.2636,沪深300ETF,4.015,18.96,2022-12-28,2022-12-28
...,...,...,...,...,...,...,...,...,...,...,...,...
281,10004167,50ETF购5月2900,0.0218,0.0218,0.000,19.29,0.0234,上证50ETF,2.793,19.97,2022-05-25,2022-05-25
282,10004166,50ETF购5月2850,0.0368,0.0368,0.000,19.43,0.0383,上证50ETF,2.793,19.97,2022-05-25,2022-05-25
283,10004165,50ETF购5月2800,0.0581,0.0581,0.000,19.60,0.0591,上证50ETF,2.793,19.97,2022-05-25,2022-05-25
284,10004164,50ETF购5月2750,0.0860,0.0430,0.043,19.78,0.0865,上证50ETF,2.793,19.97,2022-05-25,2022-05-25


In [90]:
# get code from contract number, e.g. 510050C2109M03000 => 10003371.
def get_code(contract):
    if len(contract) != 17:
        raise Exception('Not a valid contract number.')
    
    spot_name = contract[3:6].lstrip('0') + 'ETF'
    type = '购' if contract[6] == 'C' else '沽'
    month = contract[9:11].lstrip('0')
    strike = contract[-5:].lstrip('0')
    name = f'{spot_name}{type}{month}月{strike}'
    found = active_df.loc[active_df['期权名称']==name, ['期权代码', 'expire']]
    print(found)
    if len(found) == 0:
        raise Exception(f'code of {contract} not found.')
    return found.values[0]

In [91]:
get_code('510050C2105M03000')

         期权代码     expire
279  10004169 2022-05-25


array(['10004169', Timestamp('2022-05-25 00:00:00')], dtype=object)

In [92]:
lookup_df = pd.read_csv('past-50ETF-options-lookup.csv')
lookup_df['exp'] = pd.to_datetime(lookup_df['expiry'], format='%Y/%m/%d')
lookup_df

,id,num,name,type,expiry,code,exp
0,1,10004210,50ETF沽4月2450,put,2022/4/27,510050P2204M02450,2022-04-27
1,2,10004209,50ETF购4月2450,call,2022/4/27,510050C2204M02450,2022-04-27
2,3,10004134,50ETF沽4月2600,put,2022/4/27,510050P2204M02600,2022-04-27
3,4,10004133,50ETF沽4月2550,put,2022/4/27,510050P2204M02550,2022-04-27
4,5,10004132,50ETF沽4月2500,put,2022/4/27,510050P2204M02500,2022-04-27
...,...,...,...,...,...,...,...
3031,"3,032",10000005,50ETF购3月2400,call,2015/3/25,510050C1503M02400,2015-03-25
3032,"3,033",10000004,50ETF购3月2350,call,2015/3/25,510050C1503M02350,2015-03-25
3033,"3,034",10000003,50ETF购3月2300,call,2015/3/25,510050C1503M02300,2015-03-25
3034,"3,035",10000002,50ETF购3月2250,call,2015/3/25,510050C1503M02250,2015-03-25


In [93]:
import akshare as ak
[ x for x in dir(ak) if 'option' in x]

['option',
 'option_cffex_hs300_daily_sina',
 'option_cffex_hs300_list_sina',
 'option_cffex_hs300_spot_sina',
 'option_commodity_contract_sina',
 'option_commodity_contract_table_sina',
 'option_commodity_hist_sina',
 'option_current_em',
 'option_czce_daily',
 'option_czce_hist',
 'option_dce_daily',
 'option_finance_board',
 'option_finance_minute_sina',
 'option_finance_underlying',
 'option_lhb_em',
 'option_premium_analysis_em',
 'option_risk_analysis_em',
 'option_shfe_daily',
 'option_sse_codes_sina',
 'option_sse_daily_sina',
 'option_sse_expire_day_sina',
 'option_sse_greeks_sina',
 'option_sse_list_sina',
 'option_sse_minute_sina',
 'option_sse_spot_price_sina',
 'option_sse_underlying_spot_price_sina',
 'option_value_analysis_em']

In [94]:
df = ak.option_sse_daily_sina(symbol='10004206')
df.rename(columns={'收盘':'T'}, inplace=True)
df['date'] = pd.to_datetime(df['日期'])
df

,日期,开盘,最高,最低,T,成交量,date
0,2022-03-30,0.0241,0.0241,0.0126,0.0126,10264326,2022-03-30
1,2022-03-31,0.0130,0.0148,0.0128,0.0136,5654121,2022-03-31
2,2022-04-01,0.0152,0.0167,0.0088,0.0090,5739657,2022-04-01
3,2022-04-06,0.0107,0.0107,0.0073,0.0080,4419860,2022-04-06
4,2022-04-07,0.0095,0.0120,0.0070,0.0113,3721261,2022-04-07
5,2022-04-08,0.0109,0.0131,0.0080,0.0082,7434389,2022-04-08
6,2022-04-11,0.0086,0.0211,0.0086,0.0199,29737007,2022-04-11
7,2022-04-12,0.0199,0.0233,0.0113,0.0134,22444123,2022-04-12
8,2022-04-13,0.0152,0.0157,0.0110,0.0128,6703089,2022-04-13
9,2022-04-14,0.0117,0.0117,0.0070,0.0080,11237068,2022-04-14


In [95]:
ak.option_risk_analysis_em()

,期权代码,期权名称,最新价,涨跌幅,杠杆比率,实际杠杆比率,Delta,Gamma,Vega,Rho,Theta,到期日
0,10004184,300ETF购5月4200,0.0246,103.31,163.21,33.80,0.2071,1.3787,0.3121,0.0597,-0.4173,2022-05-25
1,10004183,300ETF购5月4100,0.0526,100.76,76.33,27.72,0.3632,1.8101,0.4098,0.1042,-0.5551,2022-05-25
2,10003954,300ETF购6月4900,0.0038,90.00,1056.58,4.65,0.0044,0.0432,0.0199,0.0026,-0.0128,2022-06-22
3,10003691,300ETF购6月4923A,0.0038,90.00,1056.58,3.80,0.0036,0.0365,0.0168,0.0021,-0.0108,2022-06-22
4,10003949,300ETF购6月4400,0.0266,87.32,150.94,18.48,0.1224,0.6857,0.3162,0.0715,-0.2090,2022-06-22
...,...,...,...,...,...,...,...,...,...,...,...,...
281,10004200,300ETF沽5月3800,0.0303,-46.94,132.51,17.35,-0.1309,1.0259,0.2322,-0.0398,-0.2866,2022-05-25
282,10004208,300ETF沽5月3600,0.0112,-48.62,358.48,5.34,-0.0149,0.1832,0.0415,-0.0045,-0.0518,2022-05-25
283,10004204,300ETF沽5月3700,0.0180,-49.72,223.06,11.29,-0.0506,0.5030,0.1139,-0.0153,-0.1417,2022-05-25
284,10004227,300ETF沽5月3400,0.0044,-51.11,912.50,0.37,-0.0004,0.0090,0.0020,-0.0001,-0.0025,2022-05-25


In [96]:
current_months = ak.option_sse_list_sina(symbol="50ETF", exchange="null")
current_months

['202205', '202206', '202209', '202212']

In [97]:
spot_daily_df = ak.fund_etf_hist_sina(symbol="sh510050")
spot_daily_df['date'] = pd.to_datetime(spot_daily_df['date'])
spot_daily_df

,date,open,high,low,close,volume
0,2005-02-23,0.881,0.882,0.866,0.876,1269742542
1,2005-02-24,0.876,0.876,0.868,0.876,451614223
2,2005-02-25,0.877,0.887,0.875,0.880,506460695
3,2005-02-28,0.878,0.879,0.870,0.872,187965193
4,2005-03-01,0.870,0.873,0.865,0.867,208094456
...,...,...,...,...,...,...
4175,2022-04-25,2.740,2.758,2.660,2.662,1243437148
4176,2022-04-26,2.660,2.713,2.650,2.664,1232214896
4177,2022-04-27,2.662,2.718,2.658,2.714,1386674794
4178,2022-04-28,2.706,2.754,2.703,2.749,1080532144


In [98]:
def hist(contractCode):
    if len(contractCode) != 17:
        raise Exception('not a valid contract number.')
    
    symbol, expire = '', None
    month = '20' + contractCode[7:11]
    if month in current_months:
        symbol, expire = get_code(contractCode)
    else:
        symbol, expire = lookup_df.loc[lookup_df['code']==contractCode, ['num', 'exp']].values[0]
    k = int(contractCode[-4:])/1000
    type = contractCode[6]
    print(f'symbol:{symbol}, expiredate: {expire}, strike: {k}')
    
    # get daily kline for the contract
    df = ak.option_sse_daily_sina(symbol=str(symbol))
    df.rename(columns={'收盘':'T'}, inplace=True)
    df['date'] = pd.to_datetime(df['日期'])
    
    # merge with underlying (ETF50) daily kline
    merged = pd.merge(df, spot_daily_df, how='left', on='date')
    merged['intrisicVal'] = merged['close'].map(lambda x: max(0, (x - k) * (1 if type == 'C' else -1)))
    merged['timeVal'] = merged['T'] - merged['intrisicVal']
    merged['tte'] = merged['date'].map(lambda x: (expire - x).days)
    merged['s_diff'] = merged['close'].diff()
    merged['t_diff'] = merged['T'].diff()
    greeks_df = merged.apply( 
        lambda x: [round(x, 3) for x in greeks(x['close'], k, historical_iv, interest_rate, x['tte']/365, type)] if x['tte'] > 0 else np.nan, 
        axis=1,
        result_type='expand')
    merged = merged.join(greeks_df)
    iv_func = call_iv if type == 'C' else put_iv
    merged['iv'] = merged.apply(
        lambda x: round(iv_func(x['T'], x['close'], k, x['tte']/365), 3) if x['tte'] > 0 else np.nan, 
        axis=1)
    merged.rename(columns={0:'delta',1:'gamma',2:'theta',3:'vega', 'close':'S'}, inplace=True)
    merged = merged.reindex(columns=['date', 'T', 'S', 'intrisicVal', 'timeVal', 'delta', 'gamma', 'theta', 'vega', 'iv', 's_diff', 't_diff'])
    merged.set_index('date', inplace=True)
    return merged

In [99]:
hist('510050P2206M02800')

         期权代码     expire
138  10004024 2022-06-22
symbol:10004024, expiredate: 2022-06-22 00:00:00, strike: 2.8


,T,S,intrisicVal,timeVal,delta,gamma,theta,vega,iv,s_diff,t_diff
date,,,,,,,,,,,
2022-02-07,0.0338,3.095,0.000,0.0338,-0.218,0.598,-0.174,0.555,0.198,NaN,NaN
2022-02-08,0.0302,3.098,0.000,0.0302,-0.216,0.596,-0.174,0.550,0.192,0.003,-0.0036
2022-02-09,0.0248,3.120,0.000,0.0248,-0.203,0.572,-0.170,0.531,0.187,0.022,-0.0054
2022-02-10,0.0237,3.128,0.000,0.0237,-0.197,0.564,-0.169,0.523,0.187,0.008,-0.0011
2022-02-11,0.0221,3.123,0.000,0.0221,-0.200,0.571,-0.171,0.523,0.182,-0.005,-0.0016
2022-02-14,0.0278,3.088,0.000,0.0278,-0.219,0.616,-0.179,0.540,0.186,-0.035,0.0057
2022-02-15,0.0252,3.100,0.000,0.0252,-0.211,0.603,-0.177,0.528,0.185,0.012,-0.0026
2022-02-16,0.0218,3.116,0.000,0.0218,-0.201,0.585,-0.174,0.514,0.182,0.016,-0.0034
2022-02-17,0.0216,3.119,0.000,0.0216,-0.198,0.583,-0.174,0.509,0.183,0.003,-0.0002


In [100]:
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
contract = '510050C2209M03000'
df = hist(contract)

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, specs=[[{'secondary_y':True}], [{'secondary_y':True}]])
timeval_tr = go.Bar(x=df.index, y=df['timeVal'], name='TimeValue')
intrval_tr = go.Bar(x=df.index, y=df['intrisicVal'], name='IntrisicValue')
delta_tr = go.Line(x=df.index, y=df['delta'], name='delta')
iv_tr = go.Line(x=df.index, y=df['iv'], name='iv')
theta_tr = go.Line(x=df.index, y=df['theta'], name='theta')
fig.add_traces([delta_tr, iv_tr, theta_tr], rows=1, cols=1, secondary_ys=[False, False])
spot_tr = go.Line(x=df.index, y=df['S'], name='Spot')
option_tr = go.Line(x=df.index, y=df['T'], name='Option')
fig.add_traces([spot_tr, option_tr, timeval_tr, intrval_tr], rows=2, cols=1, secondary_ys=[True, False, False, False])
fig.update_layout(barmode='stack', height=800, title=contract)
fig.show()

         期权代码     expire
104  10003978 2022-09-28
symbol:10003978, expiredate: 2022-09-28 00:00:00, strike: 3.0


C:\Users\KK\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


